In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time
import datetime
import re
import pandas as pd
import os
from selenium.common.exceptions import InvalidSessionIdException




def scrape_sephora_product(keyword="cleanser", max_pages=5):
    # **手动指定 chromedriver 位置**
    CHROMEDRIVER_PATH = "/usr/local/bin/chromedriver"

    # **配置 Selenium WebDriver**
    options = webdriver.ChromeOptions()
    options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"

    # 🚀 伪装 Selenium
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])

    # 🚀 伪装 User-Agent
    USER_AGENTS = [
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.85 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.70 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.224 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2) AppleWebKit/537.36 (KHTML, like Gecko) Version/17.0 Safari/537.36",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:120.0) Gecko/20100101 Firefox/120.0",
        "Mozilla/5.0 (Linux; Android 14; Pixel 8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6099.130 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_2 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Version/17.2 Mobile/15E148 Safari/537.36",
        "Mozilla/5.0 (iPad; CPU OS 17_2 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Version/17.2 Mobile/15E148 Safari/537.36",
        "Mozilla/5.0 (Linux; Android 13; SM-S908U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6099.130 Mobile Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0"
    ]
    random_ua = random.choice(USER_AGENTS)
    options.add_argument(f"user-agent={random_ua}")

    # 🚀 伪装真实用户行为
    options.add_argument("--start-maximized")
    options.add_argument("--window-size=1920,1080")

    # 🚀 启动 WebDriver
    service = Service(CHROMEDRIVER_PATH)
    print("✅ 启动 WebDriver...")
    driver = webdriver.Chrome(service=service, options=options)

    # **访问 Sephora**
    print("✅ 访问 Sephora...")
    url = f"https://www.sephora.com/search?keyword={keyword}"
    driver.get(url)
    time.sleep(random.uniform(1, 3))  # **延迟访问**

    # 🚀 **模拟滚动，确保 Lazy Load 数据加载完成**
    def scroll_and_wait():
        last_height = driver.execute_script("return document.body.scrollHeight")
        for _ in range(7):  # 滚动 5 次
            driver.execute_script("window.scrollBy(0, 800);")
            time.sleep(random.uniform(1, 3))
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:  # 滚动到底
                break
            last_height = new_height

    scroll_and_wait()

    # 🚀 **等待页面加载**
    wait = WebDriverWait(driver, 20)

    # 🚀 **自动点击 "Show More Results" 直到达到设定的页数**
    def click_show_more_results(max_page=4):
        pages_clicked = 0
        while pages_clicked < max_page:
            try:
                show_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Show More Products')]"))
                )
                show_more_button.click()
                print(f"✅ 点击 'Show More Results' 按钮，当前第 {pages_clicked + 1} 页")
                time.sleep(random.uniform(1, 2))  # **增加随机延迟，模拟真人操作**
                pages_clicked += 1
            except Exception as e:
                print("❌ 没有找到 'Show More Results' 按钮或已经到达最后一页")
                break

    # **调用自动翻页**
    click_show_more_results(max_page= max_pages)  # 这里你可以设置最大翻页次数

    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "css-6qisu3")))
        print("✅ 商品列表加载完成")
    except Exception as e:
        print("❌ 超时错误，未找到商品列表", e)

        driver.quit()
        exit()

    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-comp*="ProductTile"]'))
    )


    # **抓取商品信息**
    print("✅ 抓取商品数据...")
    products = driver.find_elements(By.CSS_SELECTOR, "div.css-6qisu3 > div")

    print(f"🔍 找到 {len(products)} 个商品数据")


    product_list = []

    # 在商品页提取所有商品链接
    products = driver.find_elements(By.CSS_SELECTOR, "div.css-6qisu3 > div")
    product_links = []
    for product in products:
        try:
            # 在商品列表页抓取基本信息
            product_name = product.find_element(By.CLASS_NAME, "css-1ma869u").text
            brand = product.find_element(By.CSS_SELECTOR, "span.css-1e2863e").text
            current_price = product.find_element(By.CLASS_NAME, "css-0").text
            review_count = product.find_element(By.CLASS_NAME, "css-qbbayi").text
            try:
                rate_element = product.find_element(By.XPATH, "./following-sibling::div//span[contains(@class, 'css-1jeju26')]")
                star_rating = rate_element.get_attribute("aria-label")
            except:
                star_rating = "N/A"
            
            
            try:
                # 折扣标签存在，说明有折扣
                discount_element = product.find_element(By.CSS_SELECTOR, 'span.css-rei1p9')
                is_discounted = 1
            except:
                is_discounted = 0

            # 获取商品链接
            link = product.find_element(By.TAG_NAME, "a").get_attribute("href")

            # 存储信息 + 链接备用
            product_links.append({
                "link": link,
                "product_name": product_name,
                "brand": brand,
                "price": current_price,
                "star_rating": star_rating,
                "review_count": review_count,
                "is_discounted": is_discounted,
            })

        except Exception as e:
            continue

    for item in product_links:
        try:
            # 防止打开太快被ban
            time.sleep(random.uniform(2, 3))

            # 打开新标签页并切换进去
            driver.execute_script("window.open(arguments[0]);", item["link"])
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            driver.switch_to.window(driver.window_handles[-1])

            final_url = driver.current_url

            # 等待目标元素加载完成
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.css-9b2jyy"))
            )
            unit_price = driver.find_element(By.CSS_SELECTOR, "span.css-9b2jyy").text.strip()
            # 🎯 检查送达是否三天内
            try:
                delivery_text = driver.find_element(By.CSS_SELECTOR, "span.css-13dvxca").text.strip()
                match = re.search(r"[A-Za-z]{3} \d{1,2}", delivery_text)
                if match:
                    date_str = match.group()
                    year = datetime.datetime.now().year
                    delivery_date = datetime.datetime.strptime(f"{date_str} {year}", "%b %d %Y").date()
                    today = datetime.date.today()
                    is_fast_delivery = ((delivery_date - today).days <= 3).astype(int)
                else:
                    is_fast_delivery = 0
            except:
                is_fast_delivery = 0

            # 保存到最终结果
            product_list.append({
                "product_name": item["product_name"],
                "Brand": item["brand"],
                "price": item["price"],
                "Unit_price": unit_price,
                "star_rating": item["star_rating"],
                "review_count": item["review_count"],
                "is_on_sale": is_discounted,
                "is_free_shipping": "free if sign in",
                "is_3day_delivery": is_fast_delivery,
                "Product_category": keyword,
                "Product_link": final_url,
                "platform": "Sephora"
            })

        except Exception as e:
            continue

        finally:
            try:
                if len(driver.window_handles) > 1:
                    try:
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                    except Exception as e:
                        print("⚠️ 无法切换标签页：", e)
            except InvalidSessionIdException:
                print("⚠️ WebDriver 会话已关闭，跳过标签页处理")
    
    # 创建 DataFrame
    df = pd.DataFrame(product_list)

    # 自动保存为 CSV 文件，以产品类别命名
    file_name = f"sephora_{keyword}_data.csv"
    df.to_csv(file_name, index=False)

    print(f"✅ 数据已保存为 {file_name}")

    # **关闭 WebDriver**
    driver.quit()
    print("✅ WebDriver 关闭")

    return product_list


In [2]:
#Cleanser
scrape_sephora_product(keyword="cleanser", max_pages=6)

✅ 启动 WebDriver...


KeyboardInterrupt: 

In [29]:
#Sunscreen
scrape_sephora_product(keyword="sunscreen", max_pages=6)

✅ 启动 WebDriver...
✅ 访问 Sephora...
✅ 点击 'Show More Results' 按钮，当前第 1 页
✅ 点击 'Show More Results' 按钮，当前第 2 页
✅ 点击 'Show More Results' 按钮，当前第 3 页
❌ 没有找到 'Show More Results' 按钮或已经到达最后一页
✅ 商品列表加载完成
✅ 抓取商品数据...
🔍 找到 247 个商品数据
✅ 数据已保存为 sephora_sunscreen_data.csv
✅ WebDriver 关闭


[{'product_name': 'Daily UV Defense Invisible Broad Spectrum SPF 36 Sunscreen',
  'Brand': 'innisfree',
  'price': '$18.00',
  'Unit_price': '1.69 oz / 50 ml',
  'star_rating': '5 stars',
  'review_count': '3.1K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'sunscreen',
  'Product_link': 'https://www.sephora.com/product/innisfree-daily-uv-defense-sunscreen-spf-36-P456392?skuId=2742419&icid2=products%20grid:p456392:product',
  'platform': 'Sephora'},
 {'product_name': 'Unseen Sunscreen Invisible Broad Spectrum SPF 50 PA +++',
  'Brand': 'Supergoop!',
  'price': '$22.00 - $48.00',
  'Unit_price': '1.7 oz / 50 mL',
  'star_rating': '4.5 stars',
  'review_count': '456',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'sunscreen',
  'Product_link': 'https://www.sephora.com/product/supergoop-unseen-sunscreen-invisible-broad-spectrum-spf-50-pa-P514631?skuId=2837938&icid2=p

In [30]:
#Lip Care
scrape_sephora_product(keyword="lipcare", max_pages=6)

✅ 启动 WebDriver...
✅ 访问 Sephora...
❌ 没有找到 'Show More Results' 按钮或已经到达最后一页
✅ 商品列表加载完成
✅ 抓取商品数据...
🔍 找到 32 个商品数据
✅ 数据已保存为 sephora_lipcare_data.csv
✅ WebDriver 关闭


[{'product_name': 'Le Baume Revitalizing Multi-Use Balm',
  'Brand': 'DIOR',
  'price': '$60.00',
  'Unit_price': 'Original',
  'star_rating': '4.5 stars',
  'review_count': '1K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'lipcare',
  'Product_link': 'https://www.sephora.com/product/le-baume-revitalizing-multi-use-balm-P506710?skuId=2672426&icid2=products%20grid:p506710:product',
  'platform': 'Sephora'},
 {'product_name': 'The Nip and Lip Balm - For Dry, Cracked Nipples and Lips',
  'Brand': 'Luna Daily',
  'price': '$28.00',
  'Unit_price': '0.85 oz / 25 ml',
  'star_rating': '4.5 stars',
  'review_count': '78',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'lipcare',
  'Product_link': 'https://www.sephora.com/product/luna-daily-the-nip-lip-balm-for-dry-cracked-nipples-lips-P512278?skuId=2781615&icid2=products%20grid:p512278:product',
  'platform': 'Sephora'},

In [31]:
#Face Mask
scrape_sephora_product(keyword="facemask", max_pages=6)

✅ 启动 WebDriver...
✅ 访问 Sephora...
✅ 点击 'Show More Results' 按钮，当前第 1 页
✅ 点击 'Show More Results' 按钮，当前第 2 页
❌ 没有找到 'Show More Results' 按钮或已经到达最后一页
✅ 商品列表加载完成
✅ 抓取商品数据...
🔍 找到 167 个商品数据
✅ 数据已保存为 sephora_facemask_data.csv
✅ WebDriver 关闭


[{'product_name': 'Booster Face Mask Brighten + Glow',
  'Brand': 'SEPHORA COLLECTION',
  'price': '$6.00',
  'Unit_price': '1 mask / pack',
  'star_rating': '4.5 stars',
  'review_count': '70',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'facemask',
  'Product_link': 'https://www.sephora.com/product/sephora-collection-booster-face-mask-brighten-glow-P512821?skuId=2743284&icid2=products%20grid:p512821:product',
  'platform': 'Sephora'},
 {'product_name': 'Benefiance WrinkleResist24 Pure Retinol Express Smoothing Eye Mask',
  'Brand': 'Shiseido',
  'price': '$20.00 - $72.00',
  'Unit_price': '3 Packettes x 2 Sheets',
  'star_rating': '4.5 stars',
  'review_count': '532',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'facemask',
  'Product_link': 'https://www.sephora.com/product/benefiance-wrinkleresist24-pure-retinol-express-soothing-eye-mask-P173619?icid2=products

In [38]:
#Moisturizer
scrape_sephora_product(keyword="moisturizer", max_pages=12)

✅ 启动 WebDriver...
✅ 访问 Sephora...
✅ 点击 'Show More Results' 按钮，当前第 1 页
✅ 点击 'Show More Results' 按钮，当前第 2 页
✅ 点击 'Show More Results' 按钮，当前第 3 页
✅ 点击 'Show More Results' 按钮，当前第 4 页
✅ 点击 'Show More Results' 按钮，当前第 5 页
✅ 点击 'Show More Results' 按钮，当前第 6 页
✅ 点击 'Show More Results' 按钮，当前第 7 页
✅ 点击 'Show More Results' 按钮，当前第 8 页
✅ 点击 'Show More Results' 按钮，当前第 9 页
✅ 点击 'Show More Results' 按钮，当前第 10 页
✅ 点击 'Show More Results' 按钮，当前第 11 页
✅ 点击 'Show More Results' 按钮，当前第 12 页
✅ 商品列表加载完成
✅ 抓取商品数据...
🔍 找到 782 个商品数据
✅ 数据已保存为 sephora_moisturizer_data.csv
✅ WebDriver 关闭


[{'product_name': 'The Dewy Skin Cream Plumping & Hydrating Refillable Moisturizer',
  'Brand': 'Tatcha',
  'price': '$25.00 - $89.00',
  'Unit_price': '1.7 oz / 50 mL',
  'star_rating': '4.5 stars',
  'review_count': '4.1K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'moisturizer',
  'Product_link': 'https://www.sephora.com/product/the-dewy-skin-cream-P441101?skuId=2785434&icid2=products%20grid:p441101:product',
  'platform': 'Sephora'},
 {'product_name': 'Instant Angel Lipid & Peptide-Rich Moisturizer + Barrier Repair',
  'Brand': 'Dieux',
  'price': '$45.00 - $78.00',
  'Unit_price': '1.7 oz',
  'star_rating': '4 stars',
  'review_count': '1.6K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'moisturizer',
  'Product_link': 'https://www.sephora.com/product/dieux-instant-angel-lipid-rich-barrier-repair-cream-with-ceramides-P510288?skuId=2767168&icid2=products%2

In [39]:
#Eye cream
scrape_sephora_product(keyword="eyecream", max_pages=8)

✅ 启动 WebDriver...
✅ 访问 Sephora...
✅ 点击 'Show More Results' 按钮，当前第 1 页
✅ 点击 'Show More Results' 按钮，当前第 2 页
✅ 点击 'Show More Results' 按钮，当前第 3 页
✅ 点击 'Show More Results' 按钮，当前第 4 页
❌ 没有找到 'Show More Results' 按钮或已经到达最后一页
✅ 商品列表加载完成
✅ 抓取商品数据...
🔍 找到 286 个商品数据
✅ 数据已保存为 sephora_eyecream_data.csv
✅ WebDriver 关闭


[{'product_name': 'Caffeine De-puffing + Dark Circle Eye Cream',
  'Brand': 'The INKEY List',
  'price': '$11.00',
  'Unit_price': '0.5 oz/ 15 mL',
  'star_rating': '4 stars',
  'review_count': '3K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'eyecream',
  'Product_link': 'https://www.sephora.com/product/caffeine-eye-cream-P443840?skuId=2211514&icid2=products%20grid:p443840:product',
  'platform': 'Sephora'},
 {'product_name': 'Mini Hydrating Eye Treatment Cream with Avocado',
  'Brand': "Kiehl's Since 1851",
  'price': '$38.00',
  'Unit_price': '0.5 oz/ 14 g',
  'star_rating': '4.5 stars',
  'review_count': '2.4K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'eyecream',
  'Product_link': 'https://www.sephora.com/product/creamy-eye-treatment-with-avocado-mini-P435805?skuId=1988815&icid2=products%20grid:p435805:product',
  'platform': 'Sephora'},
 {'product_name

In [41]:
#Cleansing Oil
scrape_sephora_product(keyword="cleansingoil", max_pages=8)

✅ 启动 WebDriver...
✅ 访问 Sephora...
✅ 点击 'Show More Results' 按钮，当前第 1 页
✅ 点击 'Show More Results' 按钮，当前第 2 页
✅ 点击 'Show More Results' 按钮，当前第 3 页
✅ 点击 'Show More Results' 按钮，当前第 4 页
✅ 点击 'Show More Results' 按钮，当前第 5 页
❌ 没有找到 'Show More Results' 按钮或已经到达最后一页
✅ 商品列表加载完成
✅ 抓取商品数据...
🔍 找到 361 个商品数据
✅ 数据已保存为 sephora_cleansingoil_data.csv
✅ WebDriver 关闭


[{'product_name': 'Precleanse Cleansing Oil',
  'Brand': 'Dermalogica',
  'price': '$15.00 - $77.00',
  'Unit_price': '5.1 oz/ 150 mL',
  'star_rating': '4 stars',
  'review_count': '1.4K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'cleansingoil',
  'Product_link': 'https://www.sephora.com/product/precleanse-P423148?skuId=2002079&icid2=products%20grid:p423148:product',
  'platform': 'Sephora'},
 {'product_name': 'Antioxidant Makeup Removing Cleansing Oil + Squalane',
  'Brand': 'Biossance',
  'price': '$32.00',
  'Unit_price': '6.76 oz / 200 mL',
  'star_rating': '4.5 stars',
  'review_count': '755',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'cleansingoil',
  'Product_link': 'https://www.sephora.com/product/squalane-antioxidant-cleansing-oil-P416560?skuId=2051902&icid2=products%20grid:p416560:product',
  'platform': 'Sephora'},
 {'product_name': 'Water Bank 

In [42]:
#Toner
scrape_sephora_product(keyword="toner", max_pages=8)

✅ 启动 WebDriver...
✅ 访问 Sephora...
✅ 点击 'Show More Results' 按钮，当前第 1 页
❌ 没有找到 'Show More Results' 按钮或已经到达最后一页
✅ 商品列表加载完成
✅ 抓取商品数据...
🔍 找到 123 个商品数据
✅ 数据已保存为 sephora_toner_data.csv
✅ WebDriver 关闭


[{'product_name': 'Cream Skin Refillable Toner & Moisturizer with Ceramides and Peptides',
  'Brand': 'LANEIGE',
  'price': '$6.00 - $48.00',
  'Unit_price': '5.7 oz / 170 ml',
  'star_rating': '5 stars',
  'review_count': '1K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'toner',
  'Product_link': 'https://www.sephora.com/product/cream-skin-refillable-toner-moisturizer-with-ceramides-peptides-P505897?skuId=2671857&icid2=products%20grid:p505897:product',
  'platform': 'Sephora'},
 {'product_name': 'Dynamic Gentle Resurfacing Pads with Lactic Acid',
  'Brand': 'Elemis',
  'price': '$65.00',
  'Unit_price': '60 pads',
  'star_rating': '4.5 stars',
  'review_count': '2.3K',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'toner',
  'Product_link': 'https://www.sephora.com/product/elemis-dynamic-gentle-resurfacing-pads-with-lactic-acid-P510338?icid2=products%20grid:p510

In [43]:
#Hand Cream
scrape_sephora_product(keyword="handcream", max_pages=8)

✅ 启动 WebDriver...
✅ 访问 Sephora...
❌ 没有找到 'Show More Results' 按钮或已经到达最后一页
✅ 商品列表加载完成
✅ 抓取商品数据...
🔍 找到 45 个商品数据
✅ 数据已保存为 sephora_handcream_data.csv
✅ WebDriver 关闭


[{'product_name': 'Theraplush “Hold” Refillable Overnight Retinol Hand Repair Cream',
  'Brand': 'Soft Services',
  'price': '$36.00 - $62.00',
  'Unit_price': '1.7 fl oz / 50 mL',
  'star_rating': '4.5 stars',
  'review_count': '105',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'handcream',
  'Product_link': 'https://www.sephora.com/product/theraplush-warm-hug-refillable-hand-cream-P514406?skuId=2820793&icid2=products%20grid:p514406:product',
  'platform': 'Sephora'},
 {'product_name': 'Therabu™ Buttered Reparative Hand Cream',
  'Brand': 'Drunk Elephant',
  'price': '$24.00',
  'Unit_price': '2.5 oz / 75 ml',
  'star_rating': '4.5 stars',
  'review_count': '121',
  'is_on_sale': 0,
  'is_free_shipping': 'free if sign in',
  'is_3day_delivery': 0,
  'Product_category': 'handcream',
  'Product_link': 'https://www.sephora.com/product/therabu-tm-buttered-reparative-hand-cream-P514944?icid2=products%20grid:p514944:product',
  

Clean the data

In [47]:
import pandas as pd
import glob

# 所有CSV文件
csv_files = glob.glob("sephora_*_data.csv")

# 正确列顺序
final_columns = [
    'product_name', 'brand', 'current_price', 'unit_price', 'star_rating', 'review_counts',
    'is_on_sale', 'is_free_shipping', 'is_3day_delivery', 'product_category', 'product_link', 'platform'
]

# 明确映射你刚刚给的列名 → 标准列名
rename_map = {
    'Brand': 'brand',
    'price': 'current_price',
    'Unit_price': 'unit_price',
    'star_rating': 'star_rating',
    'review_count': 'review_counts',
    'Product_category': 'product_category',
    'Product_link': 'product_link'
}

for file in csv_files:
    print(f"🔍 正在处理：{file}")
    df = pd.read_csv(file)

    # 统一列名（保持原大小写方便匹配）
    df.rename(columns=rename_map, inplace=True)

    # 统一顺序（缺的列补 NaN）
    df = df.reindex(columns=final_columns)

    # 保存文件
    output_file = file.replace(".csv", "_renamed.csv")
    df.to_csv(output_file, index=False)
    print(f"✅ 已完成：{output_file}")

🔍 正在处理：sephora_sunscreen_data.csv
✅ 已完成：sephora_sunscreen_data_renamed.csv
🔍 正在处理：sephora_cleanser_data.csv
✅ 已完成：sephora_cleanser_data_renamed.csv
🔍 正在处理：sephora_cleansingoil_data.csv
✅ 已完成：sephora_cleansingoil_data_renamed.csv
🔍 正在处理：sephora_toner_data.csv
✅ 已完成：sephora_toner_data_renamed.csv
🔍 正在处理：sephora_moisturizer_data.csv
✅ 已完成：sephora_moisturizer_data_renamed.csv
🔍 正在处理：sephora_facemask_data.csv
✅ 已完成：sephora_facemask_data_renamed.csv
🔍 正在处理：sephora_handcream_data.csv
✅ 已完成：sephora_handcream_data_renamed.csv
🔍 正在处理：sephora_lipcare_data.csv
✅ 已完成：sephora_lipcare_data_renamed.csv
🔍 正在处理：sephora_eyecream_data.csv
✅ 已完成：sephora_eyecream_data_renamed.csv


In [ ]:
#Calculate the unit_price
import pandas as pd
import glob
import re

# 找到所有 *_renamed.csv 文件
csv_files = glob.glob("*_renamed.csv")

# 提取 oz 数值的函数（只抓前面的 oz 数字）
def extract_oz(text):
    if isinstance(text, str):
        match = re.search(r"(\d+(?:\.\d+)?)\s*oz", text.lower())  # 允许小数
        if match:
            try:
                return float(match.group(1))
            except:
                return None
    return None

# 检查是否为“new”关键词（不区分大小写）
def contains_new(text):
    if isinstance(text, str):
        return bool(re.search(r"\bnew\b", text.lower()))
    return False

# 开始处理所有文件
for file in csv_files:
    print(f"🔍 正在处理文件：{file}")
    df = pd.read_csv(file)

    # 1️⃣ 删除 product_name 包含 "new" 的行
    df = df[~df["product_name"].apply(contains_new)]

    # 2️⃣ 删除 current_price 中包含 "-" 的行
    df = df[~df["current_price"].astype(str).str.contains("-")]

    # 3️⃣ 提取 unit_price 中的 oz 数字
    df["oz"] = df["unit_price"].apply(extract_oz)

    # 4️⃣ 确保 current_price 为数值（去除 $ 并转换为 float）
    df["current_price"] = df["current_price"].replace('[\$,]', '', regex=True)
    df["current_price"] = pd.to_numeric(df["current_price"], errors="coerce")

    # 5️⃣ 重新计算 unit_price：current_price / oz
    df["unit_price"] = df.apply(
        lambda row: round(row["current_price"] / row["oz"], 2)
        if pd.notnull(row["oz"]) and row["oz"] != 0 and pd.notnull(row["current_price"])
        else row["unit_price"],
        axis=1
    )

    # 6️⃣ 删除临时列
    df.drop(columns=["oz"], inplace=True)

    # 7️⃣ 输出为 _final.csv
    output_file = file.replace("_renamed.csv", "_final.csv")
    df.to_csv(output_file, index=False)
    print(f"✅ 已保存：{output_file}")

🔍 正在处理文件：sephora_moisturizer_data_renamed.csv
✅ 已保存：sephora_moisturizer_data_final.csv
🔍 正在处理文件：sephora_facemask_data_renamed.csv
✅ 已保存：sephora_facemask_data_final.csv
🔍 正在处理文件：sephora_toner_data_renamed.csv
✅ 已保存：sephora_toner_data_final.csv
🔍 正在处理文件：sephora_cleansingoil_data_renamed.csv
✅ 已保存：sephora_cleansingoil_data_final.csv
🔍 正在处理文件：sephora_lipcare_data_renamed.csv
✅ 已保存：sephora_lipcare_data_final.csv
🔍 正在处理文件：sephora_sunscreen_data_renamed.csv
✅ 已保存：sephora_sunscreen_data_final.csv
🔍 正在处理文件：sephora_cleanser_data_renamed.csv
✅ 已保存：sephora_cleanser_data_final.csv
🔍 正在处理文件：sephora_eyecream_data_renamed.csv
✅ 已保存：sephora_eyecream_data_final.csv
🔍 正在处理文件：sephora_handcream_data_renamed.csv
✅ 已保存：sephora_handcream_data_final.csv


<>:40: SyntaxWarning: invalid escape sequence '\$'
<>:40: SyntaxWarning: invalid escape sequence '\$'
/var/folders/6y/bptm9lr13kbdnrk3bc346dkm0000gp/T/ipykernel_1920/3661902761.py:40: SyntaxWarning: invalid escape sequence '\$'
  df["current_price"] = df["current_price"].replace('[\$,]', '', regex=True)


In [4]:
import pandas as pd
import glob

# 获取所有 *_final.csv 文件
csv_files = glob.glob("*_final.csv")

# 创建一个字典用于存储所有 DataFrame，key 是文件名
dataframes = {}

for file in csv_files:
    print(f"📂 正在读取：{file}")
    df = pd.read_csv(file)
    dataframes[file] = df  # 将 DataFrame 存入字典中，方便后续访问

📂 正在读取：sephora_sunscreen_data_final.csv
📂 正在读取：sephora_moisturizer_data_final.csv
📂 正在读取：sephora_eyecream_data_final.csv
📂 正在读取：sephora_cleansingoil_data_final.csv
📂 正在读取：sephora_lipcare_data_final.csv
📂 正在读取：sephora_facemask_data_final.csv
📂 正在读取：sephora_toner_data_final.csv
📂 正在读取：sephora_handcream_data_final.csv
📂 正在读取：sephora_cleanser_data_final.csv


In [18]:
cleanser_df = [df for name, df in dataframes.items() if "cleanser" in name.lower()][0]
cleanser_df = cleanser_df.copy()
cleanser_df['star_rating'] = cleanser_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
cleanser_df['is_free_shipping'] = 1
cleanser_df['is_free_shipping'] = cleanser_df['is_free_shipping'].astype(int)
cleanser_df['unit_price'] = pd.to_numeric(cleanser_df['unit_price'], errors='coerce')
cleanser_df = cleanser_df.dropna(subset=['unit_price'])
def parse_review_count(val):
    val = str(val).strip().upper()
    if 'K' in val:
        try:
            return int(float(val.replace('K', '')) * 1000)
        except:
            return None
    else:
        try:
            return int(val)
        except:
            return None

cleanser_df['review_counts'] = cleanser_df['review_counts'].apply(parse_review_count)
cleanser_df


,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
1,Glucoside Foaming Cleanser,The Ordinary,12.5,2.50,4.5,596,0,1,0,cleanser,https://www.sephora.com/product/the-ordinary-g...,Sephora
2,Avocado Ceramide Moisture Barrier Cleanser,Glow Recipe,28.0,5.60,4.5,580,0,1,0,cleanser,https://www.sephora.com/product/glow-recipe-av...,Sephora
4,Vinopure Pore Purifying Gel Cleanser,Caudalie,30.0,6.00,4.0,1100,0,1,0,cleanser,https://www.sephora.com/product/caudalie-vinop...,Sephora
5,Mini Rice Wash Skin-Softening Cleanser,Tatcha,18.0,10.59,5.0,3000,0,1,0,cleanser,https://www.sephora.com/product/tatcha-mini-ri...,Sephora
6,ATOBARRIER365 Gentle pH-Balancing Foaming Clea...,AESTURA,21.0,4.14,4.0,96,0,1,0,cleanser,https://www.sephora.com/product/atobarrier365-...,Sephora
...,...,...,...,...,...,...,...,...,...,...,...,...
130,Take The Day Off Cleansing Oil Makeup Remover,CLINIQUE,37.0,5.52,4.0,593,0,1,0,cleanser,https://www.sephora.com/product/take-the-day-o...,Sephora
131,Vinoclean Cleansing Micellar Water,Caudalie,30.0,4.48,3.5,687,0,1,0,cleanser,https://www.sephora.com/product/micellar-clean...,Sephora
132,Total Cleansing Oil & Makeup Remover,Clarins,39.0,7.80,4.5,8,0,1,0,cleanser,https://www.sephora.com/product/clarins-total-...,Sephora
133,Mini The POREfessional Good Cleanup Foaming Cl...,Benefit Cosmetics,15.0,10.00,5.0,501,0,1,0,cleanser,https://www.sephora.com/product/benefit-cosmet...,Sephora


In [22]:
cleansingoil_df = [df for name, df in dataframes.items() if "cleansingoil" in name.lower()][0]
cleansingoil_df = cleansingoil_df.copy()
cleansingoil_df['star_rating'] = cleansingoil_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
cleansingoil_df['is_free_shipping'] = 1
cleansingoil_df['is_free_shipping'] = cleansingoil_df['is_free_shipping'].astype(int)
cleansingoil_df['unit_price'] = pd.to_numeric(cleansingoil_df['unit_price'], errors='coerce')
cleansingoil_df = cleansingoil_df.dropna(subset=['unit_price'])
cleansingoil_df['review_counts'] = cleansingoil_df['review_counts'].apply(parse_review_count)
cleansingoil_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
0,Antioxidant Makeup Removing Cleansing Oil + Sq...,Biossance,32.0,4.73,4.5,755,0,1,0,cleansingoil,https://www.sephora.com/product/squalane-antio...,Sephora
1,Mini Gentle Cleansing Oil Makeup Remover,Sulwhasoo,13.0,7.65,4.0,872,0,1,0,cleansingoil,https://www.sephora.com/product/sulwhasoo-mini...,Sephora
2,Vinoclean Makeup Removing Cleansing Oil,Caudalie,30.0,6.00,4.5,533,0,1,0,cleansingoil,https://www.sephora.com/product/caudalie-vinoc...,Sephora
3,Barrier+ Foaming Oil Hydrating Cleanser,Skinfix,30.0,5.00,3.5,1300,0,1,0,cleansingoil,https://www.sephora.com/product/barrier-foamin...,Sephora
4,Mini Oat Cleansing Balm,The INKEY List,7.0,4.12,4.5,5300,0,1,0,cleansingoil,https://www.sephora.com/product/the-inkey-list...,Sephora
...,...,...,...,...,...,...,...,...,...,...,...,...
104,Superfood + Niacinamide Body Cleanser with Ant...,Youth To The People,39.0,2.38,4.5,514,0,1,0,cleansingoil,https://www.sephora.com/product/youth-to-the-p...,Sephora
105,NUDESKIN Citrus Clean Balm & Make-Up Melt,NUDESTIX,35.0,17.50,4.5,146,0,1,0,cleansingoil,https://www.sephora.com/product/nudestix-nudes...,Sephora
106,Beautifully Balanced Body Oil Cleanser,Isle of Paradise,22.0,2.33,4.0,205,0,1,0,cleansingoil,https://www.sephora.com/product/isle-paradise-...,Sephora
108,Black Reserve™ Body & Hair Cleanser,Jack Black,24.0,2.40,4.5,4,0,1,0,cleansingoil,https://www.sephora.com/product/black-reserve-...,Sephora


In [21]:
facemask_df = [df for name, df in dataframes.items() if "facemask" in name.lower()][0]
facemask_df = facemask_df.copy()
facemask_df['star_rating'] = facemask_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
facemask_df['is_free_shipping'] = 1
facemask_df['is_free_shipping'] = facemask_df['is_free_shipping'].astype(int)
facemask_df['unit_price'] = pd.to_numeric(facemask_df['unit_price'], errors='coerce')
facemask_df = facemask_df.dropna(subset=['unit_price'])
facemask_df['review_counts'] = facemask_df['review_counts'].apply(parse_review_count)

facemask_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
1,Cryo Rubber™ Firming Face Mask,Dr. Jart+,16.0,114.29,4.5,765,0,1,0,facemask,https://www.sephora.com/product/cryo-rubber-tm...,Sephora
2,Super Volcanic AHA Pore Clearing Clay Mask,innisfree,18.0,5.33,4.5,1900,0,1,0,facemask,https://www.sephora.com/product/innisfree-pore...,Sephora
3,Cryo Rubber™ Face Mask with Hyaluronic Acid,Dr. Jart+,16.0,114.29,4.5,765,0,1,0,facemask,https://www.sephora.com/product/cryo-rubber-tm...,Sephora
6,Mini Pore Minimizing Instant Detox Mask,Caudalie,22.0,18.64,3.0,1900,0,1,0,facemask,https://www.sephora.com/product/caudalie-mini-...,Sephora
9,Floral Recovery Overnight Mask with Squalane,fresh,72.0,21.82,4.5,1200,0,1,0,facemask,https://www.sephora.com/product/fresh-floral-r...,Sephora
...,...,...,...,...,...,...,...,...,...,...,...,...
107,Bye Bye Makeup 3-in-1 Makeup Melting Cleansing...,IT Cosmetics,47.0,11.75,4.5,368,0,1,0,facemask,https://www.sephora.com/product/it-cosmetics-b...,Sephora
108,Cold Plunge™ Pore Remedy Moisturizer with BHA/LHA,OLEHENRIKSEN,51.0,30.00,4.0,1100,0,1,0,facemask,https://www.sephora.com/product/ole-henriksen-...,Sephora
109,Clarifying Mask,Dr. Barbara Sturm,150.0,88.76,4.5,21,0,1,0,facemask,https://www.sephora.com/product/dr-barbara-stu...,Sephora
114,Skin Refining Mild PHA Toner with 8% PHA for G...,Biodance,21.0,4.14,5.0,39,0,1,0,facemask,https://www.sephora.com/product/skin-refining-...,Sephora


In [23]:
handcream_df = [df for name, df in dataframes.items() if "handcream" in name.lower()][0]
handcream_df = handcream_df.copy()
handcream_df['star_rating'] = handcream_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
handcream_df['is_free_shipping'] = 1
handcream_df['is_free_shipping'] = handcream_df['is_free_shipping'].astype(int)
handcream_df['unit_price'] = pd.to_numeric(handcream_df['unit_price'], errors='coerce')
handcream_df = handcream_df.dropna(subset=['unit_price'])
handcream_df['review_counts'] = handcream_df['review_counts'].apply(parse_review_count)
handcream_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
0,Therabu™ Buttered Reparative Hand Cream,Drunk Elephant,24.0,9.60,4.5,121,0,1,0,handcream,https://www.sephora.com/product/therabu-tm-but...,Sephora
1,Hand and Nail Cream,Caudalie,16.0,6.40,4.0,1500,0,1,0,handcream,https://www.sephora.com/product/hand-and-nail-...,Sephora
2,Find Comfort Niacinamide Hydrating Hand Cream,Rare Beauty by Selena Gomez,18.0,11.25,4.5,744,0,1,0,handcream,https://www.sephora.com/product/rare-beauty-by...,Sephora
3,Hand Cream,Glossier,20.0,10.00,4.5,108,0,1,0,handcream,https://www.sephora.com/product/glossier-hand-...,Sephora
4,The Hand Cream - Barrier Treatment with 5 Cera...,Nécessaire,28.0,12.73,5.0,41,0,1,0,handcream,https://www.sephora.com/product/necessaire-the...,Sephora
5,Protecting & Repairing Shea Butter Intensive H...,L'Occitane,40.0,7.55,4.0,299,0,1,0,handcream,https://www.sephora.com/product/shea-butter-in...,Sephora
6,Bergamot & Hinoki Nourishing Hand Cream with N...,Salt & Stone,20.0,10.00,4.5,32,0,1,0,handcream,https://www.sephora.com/product/salt-stone-ber...,Sephora
7,Black Rose & Oud Nourishing Hand Cream with Ni...,Salt & Stone,20.0,10.00,4.5,27,0,1,0,handcream,https://www.sephora.com/product/salt-stone-bla...,Sephora
8,Brazilian Touch Hand Cream,Sol de Janeiro,16.0,9.41,5.0,172,0,1,0,handcream,https://www.sephora.com/product/sol-de-janeiro...,Sephora
9,The Hand Retinol - Repair Serum with 0.25% Pur...,Nécessaire,38.0,17.27,4.5,260,0,1,0,handcream,https://www.sephora.com/product/necessaire-the...,Sephora


In [24]:
lipcare_df = [df for name, df in dataframes.items() if "lipcare" in name.lower()][0]
lipcare_df = lipcare_df.copy()
lipcare_df['star_rating'] = lipcare_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
lipcare_df['is_free_shipping'] = 1
lipcare_df['is_free_shipping'] = lipcare_df['is_free_shipping'].astype(int)
lipcare_df['unit_price'] = pd.to_numeric(lipcare_df['unit_price'], errors='coerce')
lipcare_df = lipcare_df.dropna(subset=['unit_price'])
lipcare_df['review_counts'] = lipcare_df['review_counts'].apply(parse_review_count)
lipcare_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
1,"The Nip and Lip Balm - For Dry, Cracked Nipple...",Luna Daily,28.0,32.94,4.5,78,0,1,0,lipcare,https://www.sephora.com/product/luna-daily-the...,Sephora
2,Squalane + Amino Acids Hydrating Lip Balm,The Ordinary,9.5,19.00,3.5,421,0,1,0,lipcare,https://www.sephora.com/product/the-ordinary-s...,Sephora
3,Lippe Balm,Drunk Elephant,19.0,146.15,4.5,2100,0,1,0,lipcare,https://www.sephora.com/product/lippe-tm-P4038...,Sephora
8,24-7 Moisture Day & Night Peptide Lip Mask,TULA Skincare,18.0,64.29,4.5,6,0,1,0,lipcare,https://www.sephora.com/product/tula-skincare-...,Sephora


In [25]:
moisturizer_df = [df for name, df in dataframes.items() if "moisturizer" in name.lower()][0]
moisturizer_df = moisturizer_df.copy()
moisturizer_df['star_rating'] = moisturizer_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
moisturizer_df['is_free_shipping'] = 1
moisturizer_df['is_free_shipping'] = moisturizer_df['is_free_shipping'].astype(int)
moisturizer_df['unit_price'] = pd.to_numeric(moisturizer_df['unit_price'], errors='coerce')
moisturizer_df = moisturizer_df.dropna(subset=['unit_price'])
moisturizer_df['review_counts'] = moisturizer_df['review_counts'].apply(parse_review_count)
moisturizer_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
1,Hydrating Dewy Gel Cream Moisturizer with Hyal...,First Aid Beauty,38.0,20.00,5.0,107,0,1,0,moisturizer,https://www.sephora.com/product/hydrating-dewy...,Sephora
2,ATOBARRIER365 Lightweight Face Lotion Moisturi...,AESTURA,32.0,6.31,4.5,187,0,1,0,moisturizer,https://www.sephora.com/product/atobarrier365-...,Sephora
3,Omega Water Cream Oil-Free Moisturizer + Niaci...,The INKEY List,12.0,7.06,4.5,1700,0,1,0,moisturizer,https://www.sephora.com/product/the-inkey-list...,Sephora
4,Mini Vitamin Enriched Face Base Moisturizer & ...,Bobbi Brown,25.0,50.00,4.0,4600,0,1,0,moisturizer,https://www.sephora.com/product/vitamin-enrich...,Sephora
5,Mini Jet Lag Mask + Moisturizer,Summer Fridays,26.0,26.00,4.0,5900,0,1,0,moisturizer,https://www.sephora.com/product/summer-fridays...,Sephora
...,...,...,...,...,...,...,...,...,...,...,...,...
152,Hydrating Serum with Hyaluronic Acid,First Aid Beauty,40.0,23.53,4.5,153,0,1,0,moisturizer,https://www.sephora.com/product/hydrating-seru...,Sephora
153,KP+ Psoriasis Smoothing Treatment Body Lotion,Skinfix,50.0,9.86,4.5,309,0,1,0,moisturizer,https://www.sephora.com/product/kp-pso-smoothi...,Sephora
155,Age-defying Nourishing Immortelle Youth Hand C...,L'Occitane,40.0,14.81,4.0,23,0,1,0,moisturizer,https://www.sephora.com/product/l-occitane-you...,Sephora
156,Bergamot & Hinoki Nourishing Hand Cream with N...,Salt & Stone,20.0,10.00,4.5,32,0,1,0,moisturizer,https://www.sephora.com/product/salt-stone-ber...,Sephora


In [30]:
eyecream_df = [df for name, df in dataframes.items() if "eyecream" in name.lower()][0]
eyecream_df = eyecream_df.copy()
eyecream_df['star_rating'] = eyecream_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
eyecream_df['is_free_shipping'] = 1
eyecream_df['is_free_shipping'] = eyecream_df['is_free_shipping'].astype(int)
eyecream_df['unit_price'] = pd.to_numeric(eyecream_df['unit_price'], errors='coerce')
eyecream_df = eyecream_df.dropna(subset=['unit_price'])
eyecream_df['review_counts'] = eyecream_df['review_counts'].apply(parse_review_count)
eyecream_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
0,Caffeine De-puffing + Dark Circle Eye Cream,The INKEY List,11.0,22.00,4.0,3000,0,1,0,eyecream,https://www.sephora.com/product/caffeine-eye-c...,Sephora
1,Mini Hydrating Eye Treatment Cream with Avocado,Kiehl's Since 1851,38.0,76.00,4.5,2400,0,1,0,eyecream,https://www.sephora.com/product/creamy-eye-tre...,Sephora
2,Vinoperfect Dark Circle Brightening Eye Cream ...,Caudalie,55.0,110.00,5.0,1100,0,1,0,eyecream,https://www.sephora.com/product/vinoperfect-da...,Sephora
3,Marine Algae Eye Cream for Fine Lines & Dark C...,Biossance,56.0,112.00,4.5,1400,0,1,0,eyecream,https://www.sephora.com/product/squalane-marin...,Sephora
4,The Brightening Eye Cream with Vitamin-C for D...,Tatcha,64.0,128.00,4.0,308,0,1,0,eyecream,https://www.sephora.com/product/tatcha-brighte...,Sephora
...,...,...,...,...,...,...,...,...,...,...,...,...
89,Plantscription™ Anti-Aging Power Eye Cream,Origins,66.0,132.00,4.5,133,0,1,0,eyecream,https://www.sephora.com/product/plantscription...,Sephora
91,The Quench Brightening Eye Balm with Line-Fili...,BeautyBio,99.0,19.80,3.5,9,0,1,0,eyecream,https://www.sephora.com/product/beautybio-the-...,Sephora
93,Age Reversal Eye Complex,Dermalogica,79.0,158.00,3.0,25,0,1,0,eyecream,https://www.sephora.com/product/age-reversal-e...,Sephora
94,The Lifting Eye Serum,La Mer,320.0,640.00,4.5,18,0,1,0,eyecream,https://www.sephora.com/product/the-lifting-ey...,Sephora


In [26]:
sunscreen_df = [df for name, df in dataframes.items() if "sunscreen" in name.lower()][0]
sunscreen_df = sunscreen_df.copy()
sunscreen_df['star_rating'] = sunscreen_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
sunscreen_df['is_free_shipping'] = 1
sunscreen_df['is_free_shipping'] = sunscreen_df['is_free_shipping'].astype(int)
sunscreen_df['unit_price'] = pd.to_numeric(sunscreen_df['unit_price'], errors='coerce')
sunscreen_df = sunscreen_df.dropna(subset=['unit_price'])
sunscreen_df['review_counts'] = sunscreen_df['review_counts'].apply(parse_review_count)
sunscreen_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
0,Daily UV Defense Invisible Broad Spectrum SPF ...,innisfree,18.0,10.65,5.0,3100,0,1,0,sunscreen,https://www.sephora.com/product/innisfree-dail...,Sephora
1,Clear Sunscreen Stick SPF 50,Shiseido,32.0,45.71,4.5,1800,0,1,0,sunscreen,https://www.sephora.com/product/wetforce-clear...,Sephora
2,ShadeDrops Mineral Milk Sunscreen SPF 30,Summer Fridays,36.0,21.18,4.5,1600,0,1,0,sunscreen,https://www.sephora.com/product/shadedrops-bro...,Sephora
4,Urban Environment Fresh-Moisture Sunscreen Bro...,Shiseido,30.4,16.89,4.5,1200,0,1,0,sunscreen,https://www.sephora.com/product/shiseido-urban...,Sephora
5,Hydro UV Defense Sunscreen Broad Spectrum SPF 50+,LANEIGE,30.0,17.96,5.0,201,0,1,0,sunscreen,https://www.sephora.com/product/laneige-hydro-...,Sephora
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Invisiblur™ Perfecting Face Sunscreen Broad Sp...,Murad,71.0,71.00,4.0,788,0,1,0,sunscreen,https://www.sephora.com/product/invisiblur-per...,Sephora
78,A Perfect World™ SPF 40 Age-Defense Moisturize...,Origins,58.0,34.12,5.0,365,0,1,0,sunscreen,https://www.sephora.com/product/a-perfect-worl...,Sephora
80,Mineral Sun Defense Moisturizer Broad Spectrum...,Naturally Serious,36.0,21.18,4.0,150,0,1,0,sunscreen,https://www.sephora.com/product/mineral-sun-de...,Sephora
81,Mini Moisture Surge™ Broad Spectrum SPF 28 She...,CLINIQUE,31.0,31.00,4.0,24,0,1,0,sunscreen,https://www.sephora.com/product/mini-moisture-...,Sephora


In [27]:
toner_df = [df for name, df in dataframes.items() if "toner" in name.lower()][0]
toner_df = toner_df.copy()
toner_df['star_rating'] = toner_df['star_rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
toner_df['is_free_shipping'] = 1
toner_df['is_free_shipping'] = toner_df['is_free_shipping'].astype(int)
toner_df['unit_price'] = pd.to_numeric(toner_df['unit_price'], errors='coerce')
toner_df = toner_df.dropna(subset=['unit_price'])
toner_df['review_counts'] = toner_df['review_counts'].apply(parse_review_count)
toner_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
2,Saccharomyces Ferment 30% Milky Toner,The Ordinary,14.0,4.12,4.5,894,0,1,0,toner,https://www.sephora.com/product/the-ordinary-s...,Sephora
3,Deep Sweep 2% BHA Pore Cleaning Toner with Mor...,Farmacy,34.0,8.50,4.5,1000,0,1,0,toner,https://www.sephora.com/product/farmacy-deep-s...,Sephora
4,Mini Glycolic Acid 7% Exfoliating Toner,The Ordinary,8.7,2.57,4.0,724,0,1,0,toner,https://www.sephora.com/product/the-ordinary-m...,Sephora
5,Vinopure Natural Salicylic Acid Pore Minimizin...,Caudalie,30.0,4.48,4.5,534,0,1,0,toner,https://www.sephora.com/product/caudalie-vinop...,Sephora
6,RESIST Advanced Replenishing Toner with Hyalur...,Paula's Choice,29.0,7.25,4.5,370,0,1,0,toner,https://www.sephora.com/product/paula-s-choice...,Sephora
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Clarifying Lotion 4,CLINIQUE,37.0,2.74,NaN,231,0,1,0,toner,https://www.sephora.com/product/clarifying-lot...,Sephora
76,Buffering Lotion,Mario Badescu,20.0,20.00,NaN,43,0,1,0,toner,https://www.sephora.com/product/mario-badescu-...,Sephora
77,Moisturizing High Performance Face Cream with ...,MACRENE actives,225.0,225.00,4.0,284,0,1,0,toner,https://www.sephora.com/product/macrene-active...,Sephora
78,Brownish Tinted Dry Shampoo,Bumble and bumble,43.0,9.77,3.0,512,0,1,0,toner,https://www.sephora.com/product/a-tint-of-brow...,Sephora


In [41]:

product_dfs = {
    "cleanser": cleanser_df,
    "cleansingoil": cleansingoil_df,
    "eyecream": eyecream_df,
    "facemask": facemask_df,
    "handcream": handcream_df,
    "lipcare": lipcare_df,
    "moisturizer": moisturizer_df,
    "sunscreen": sunscreen_df,
    "toner": toner_df
}

for product, df in product_dfs.items():
    output_name = f"sephora_{product}_data_cleaned.csv"
    df.to_csv(output_name, index=False)
    print(f"✅ 已保存文件：{output_name}")

✅ 已保存文件：sephora_cleanser_data_cleaned.csv
✅ 已保存文件：sephora_cleansingoil_data_cleaned.csv
✅ 已保存文件：sephora_eyecream_data_cleaned.csv
✅ 已保存文件：sephora_facemask_data_cleaned.csv
✅ 已保存文件：sephora_handcream_data_cleaned.csv
✅ 已保存文件：sephora_lipcare_data_cleaned.csv
✅ 已保存文件：sephora_moisturizer_data_cleaned.csv
✅ 已保存文件：sephora_sunscreen_data_cleaned.csv
✅ 已保存文件：sephora_toner_data_cleaned.csv


In [31]:
#Concact the dataframe
all_products_df = pd.concat([
    cleanser_df,
    cleansingoil_df,
    eyecream_df,
    handcream_df,
    lipcare_df,
    moisturizer_df,
    sunscreen_df,
    toner_df
], ignore_index=True)
all_products_df

,product_name,brand,current_price,unit_price,star_rating,review_counts,is_on_sale,is_free_shipping,is_3day_delivery,product_category,product_link,platform
0,Glucoside Foaming Cleanser,The Ordinary,12.5,2.50,4.5,596,0,1,0,cleanser,https://www.sephora.com/product/the-ordinary-g...,Sephora
1,Avocado Ceramide Moisture Barrier Cleanser,Glow Recipe,28.0,5.60,4.5,580,0,1,0,cleanser,https://www.sephora.com/product/glow-recipe-av...,Sephora
2,Vinopure Pore Purifying Gel Cleanser,Caudalie,30.0,6.00,4.0,1100,0,1,0,cleanser,https://www.sephora.com/product/caudalie-vinop...,Sephora
3,Mini Rice Wash Skin-Softening Cleanser,Tatcha,18.0,10.59,5.0,3000,0,1,0,cleanser,https://www.sephora.com/product/tatcha-mini-ri...,Sephora
4,ATOBARRIER365 Gentle pH-Balancing Foaming Clea...,AESTURA,21.0,4.14,4.0,96,0,1,0,cleanser,https://www.sephora.com/product/atobarrier365-...,Sephora
...,...,...,...,...,...,...,...,...,...,...,...,...
630,Clarifying Lotion 4,CLINIQUE,37.0,2.74,NaN,231,0,1,0,toner,https://www.sephora.com/product/clarifying-lot...,Sephora
631,Buffering Lotion,Mario Badescu,20.0,20.00,NaN,43,0,1,0,toner,https://www.sephora.com/product/mario-badescu-...,Sephora
632,Moisturizing High Performance Face Cream with ...,MACRENE actives,225.0,225.00,4.0,284,0,1,0,toner,https://www.sephora.com/product/macrene-active...,Sephora
633,Brownish Tinted Dry Shampoo,Bumble and bumble,43.0,9.77,3.0,512,0,1,0,toner,https://www.sephora.com/product/a-tint-of-brow...,Sephora


In [33]:
!pip install -qqq mysqlclient
!pip install -qqq -U sql_magic


In [34]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd
from sqlalchemy import create_engine

conn_string = 'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user="dbmh",
    password="jelIyvfzqwNBWJXpC9bJTQ==",
    host = 'jsedocc7.scrc.nyu.edu',
    port     = 3306,
    encoding = 'utf-8',
    db = 'dbmh'
)
engine = create_engine(conn_string)
pd.read_sql('SHOW TABLES', con=engine)

,Tables_in_dbmh
0,Makeup_Table
1,Products_Table


In [37]:
import pandas as pd
df_lis=[]
df_lis.append(all_products_df)

In [38]:
# **Checking database connection**
try:
    conn = engine.connect()
    print("✅ 成功连接到数据库！")
except Exception as e:
    print("❌ 数据库连接失败:", e)
    exit()

# **Checking table existence**
table_name = "Makeup_Table"
existing_tables = pd.read_sql("SHOW TABLES", con=engine)

if table_name not in existing_tables.values:
    print(f"🔍 表 {table_name} 不存在，创建新表...")
else:
    print(f"✅ 表 {table_name} 已存在，数据将追加存储！")

# **Store data into MySQL**
try:
    for df in df_lis:
        df.to_sql(table_name, con=engine, if_exists='append', index=False)
        print("✅ 数据成功存入 MySQL 数据库！")
except Exception as e:
    print("❌ 数据存储失败:", e)

# **Close the database**
conn.close()

✅ 成功连接到数据库！
✅ 表 Makeup_Table 已存在，数据将追加存储！
✅ 数据成功存入 MySQL 数据库！
